In [30]:
import veloxchem as vlx
from openmmdynamics3 import OpenMMDynamics

In [31]:
molecule = vlx.Molecule.read_smiles('COc1ccc2c(c1)nc([nH]2)S(=O)Cc1ncc(c(c1C)OC)C')
print('Structure of the molecule entered: ')
molecule.show(atom_indices=False)


Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
ff_gen = vlx.ForceFieldGenerator()
#ff_gen.ostream.mute()
ff_gen.create_topology(molecule)
ff_gen.write_openmm_files('mol','MOL')

* Info * Using 6-31G* basis set for RESP charges...                                                                       
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                

In [4]:
opm_dyn = OpenMMDynamics()
pdb_files = ['mol.pdb']
xml_files = ['mol.xml']
opm_dyn.build_custom_system(pdb_files, xml_files, phase='gas')
steps=1000000
snap=200

# Vi kan nu köra en kort simulering för att se att allt fungerar!!!
opm_dyn.run_md(ensemble='NVT', 
               temperature=700,
               timestep=2.0,
               nsteps=steps,
               snapshots=snap)

Running simulation...
Ensemble: NVT
Temperature: 700 K
Friction: 1.0 1/ps
Timestep: 2.0 fs
Total simulation time in ns: 2.0
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Density (g/mL)"
5000,316.91343688964844,755.6895706881143,0.07163455794912381
10000,284.40716552734375,656.8875630336177,0.07163455794912381
15000,368.6852264404297,667.2729025681356,0.07163455794912381
20000,275.5163269042969,813.3522946526987,0.07163455794912381
25000,356.3454895019531,769.2213942070271,0.07163455794912381
30000,316.07655334472656,732.2144913398994,0.07163455794912381
35000,389.72882080078125,807.2971786521165,0.07163455794912381
40000,425.0579376220703,752.0133032755812,0.07163455794912381
45000,322.6205749511719,690.6983081198697,0.07163455794912381
50000,312.01409912109375,711.1932456729677,0.07163455794912381
55000,337.47222900390625,726.0336223840618,0.07163455794912381
60000,221.69699096679688,631.7442139186462,0.07163455794912381
65000,338.5003662109375,730.2995785609198,0.0716345579

In [5]:
import MDAnalysis as mda

for i in range(snap):
  universe = mda.Universe('trajectory.pdb')
  universe.trajectory[i]
  allatoms = universe.select_atoms('all')
  allatoms.write('trajectory1_' + str(i) + '.pdb')
  #with mda.Writer(gromacs_file + '.xyz') as xyz:
  #   xyz.write(universe)



/opt/miniconda3/envs/vlxenv/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/opt/miniconda3/envs/vlxenv/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [20]:
opm_dyn = OpenMMDynamics()
energies1 = []
geometries1 = []
for i in range(snap):
    print('step ' + str(i+1) + ' of ' + str(snap))
    opm_dyn.build_custom_system('trajectory1_' + str(i) + '.pdb', 'mol.xml', phase='gas', topology_pdb='mol.pdb')
    energy, geometry = opm_dyn.energy_minimization()
    energies1.append(energy)
    geometries1.append(geometry)

step 1 of 200
Topology PDB file provided
Geometry optimized
step 2 of 200
Topology PDB file provided
Geometry optimized
step 3 of 200
Topology PDB file provided
Geometry optimized
step 4 of 200
Topology PDB file provided
Geometry optimized
step 5 of 200
Topology PDB file provided
Geometry optimized
step 6 of 200
Topology PDB file provided
Geometry optimized
step 7 of 200
Topology PDB file provided
Geometry optimized
step 8 of 200
Topology PDB file provided
Geometry optimized
step 9 of 200
Topology PDB file provided
Geometry optimized
step 10 of 200
Topology PDB file provided
Geometry optimized
step 11 of 200
Topology PDB file provided
Geometry optimized
step 12 of 200
Topology PDB file provided
Geometry optimized
step 13 of 200
Topology PDB file provided
Geometry optimized
step 14 of 200
Topology PDB file provided
Geometry optimized
step 15 of 200
Topology PDB file provided
Geometry optimized
step 16 of 200
Topology PDB file provided
Geometry optimized
step 17 of 200
Topology PDB file 

In [21]:
m = min(energies1)
m_index = energies1.index(m)

for i in range(snap):
  lowest_geometry = vlx.Molecule.read_xyz_string(geometries1[i])
  print('Relative energy: ', (energies1[i] - energies1[m_index]))
  #lowest_geometry.show()

Relative energy:  8.330184936523438
Relative energy:  6.786956787109375
Relative energy:  6.6153717041015625
Relative energy:  3.8927001953125
Relative energy:  0.6652984619140625
Relative energy:  6.2386322021484375
Relative energy:  5.204833984375
Relative energy:  4.5495452880859375
Relative energy:  6.8255767822265625
Relative energy:  13.194854736328125
Relative energy:  7.65301513671875
Relative energy:  10.20501708984375
Relative energy:  1.0550079345703125
Relative energy:  1.236785888671875
Relative energy:  1.9286346435546875
Relative energy:  8.470001220703125
Relative energy:  4.9961090087890625
Relative energy:  17.061721801757812
Relative energy:  6.90435791015625
Relative energy:  10.58929443359375
Relative energy:  9.563278198242188
Relative energy:  10.549072265625
Relative energy:  10.906906127929688
Relative energy:  12.6190185546875
Relative energy:  8.510772705078125
Relative energy:  5.71636962890625
Relative energy:  19.088851928710938
Relative energy:  1.8061370

In [22]:
lowest_geometryS = vlx.Molecule.read_xyz_string(geometries1[m_index])
lowest_geometryS.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [23]:
print(geometries1[m_index])

43

C 8.97064208984375 6.846248507499695 2.742163836956024
O 8.145182728767395 6.0770344734191895 1.8566049635410309
C 7.716803550720215 6.650930643081665 0.6768376380205154
C 8.133077025413513 7.9038190841674805 0.1761304959654808
C 7.633780837059021 8.419257402420044 -1.044265627861023
C 6.705537438392639 7.616597414016724 -1.7226691544055939
C 6.2862008810043335 6.373512148857117 -1.2467949837446213
C 6.794834733009338 5.879004001617432 -0.040317364037036896
N 5.396732687950134 5.800392031669617 -2.1169479191303253
C 5.263373851776123 6.649651527404785 -3.1151050329208374
N 6.0196298360824585 7.775439023971558 -2.903262972831726
S 4.270567893981934 6.323872804641724 -4.534805715084076
O 4.20497864484787 7.624881267547607 -5.291537046432495
C 5.346258282661438 5.225149393081665 -5.50726592540741
C 5.794041156768799 3.975815176963806 -4.755981862545013
N 4.845888614654541 2.9973971843719482 -4.821915924549103
C 5.0718241930007935 1.8216580152511597 -4.2041632533073425
C 6.226518750190